In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import plotly.express as px


from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM, Dropout , GRU
from tensorflow.keras.callbacks import EarlyStopping

import warnings
from warnings import filterwarnings
warnings.simplefilter(action='ignore')
%matplotlib inline

In [2]:
var_bank = 'BBCA'
df = pd.read_csv(f'/content/{var_bank}_FS.csv',parse_dates=True)
df

,Date,Bollinger_Upper,SMA_7,High,SMA_30,Low,Open,Bollinger_Lower,Close,Rolling_Mean,EMA_12,EMA_26,WMA,Adj Close,SMA_14
0,2010-05-17,1191.651805,1027.857143,1070.0,1108.166667,1010.0,1070.0,982.848195,1070.0,1087.25,1058.693294,1079.837715,1041.181818,847.906067,1067.500000
1,2010-05-18,1187.718224,1032.142857,1110.0,1107.500000,1060.0,1090.0,982.781776,1090.0,1085.25,1063.532787,1080.660585,1049.090909,863.754639,1062.500000
2,2010-05-19,1174.631505,1042.142857,1090.0,1105.500000,1040.0,1050.0,983.868495,1050.0,1079.25,1061.442386,1078.194917,1049.909091,832.057068,1058.928571
3,2010-05-20,1164.528794,1051.428571,1060.0,1102.500000,1010.0,1040.0,982.971206,1040.0,1073.75,1058.132256,1075.142715,1049.818182,824.132813,1056.071429
4,2010-05-21,1155.917007,1051.428571,1030.0,1098.166667,995.0,1020.0,979.582993,1020.0,1067.75,1052.248758,1070.761766,1046.090909,808.284180,1052.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3426,2024-03-22,10328.873676,10150.000000,10125.0,9923.333333,9950.0,10000.0,9671.126324,10100.0,10000.00,10084.187307,9974.022685,10140.454545,9872.500000,10037.500000
3427,2024-03-25,10335.010121,10164.285714,10075.0,9936.666667,9925.0,10075.0,9684.989879,10075.0,10010.00,10082.773875,9981.502486,10128.181818,10075.000000,10053.571429
3428,2024-03-26,10334.667075,10128.571429,10050.0,9953.333333,9950.0,10050.0,9707.832925,10050.0,10021.25,10077.731740,9986.576376,10112.272727,10050.000000,10071.428571
3429,2024-03-27,10331.204055,10114.285714,10075.0,9967.500000,10000.0,10025.0,9738.795945,10075.0,10035.00,10077.311473,9993.126274,10102.727273,10075.000000,10092.857143


In [3]:
DF = df.drop('Date',axis=1)
X = DF
y = DF[['Open', 'High', 'Low', 'Close']]
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()
X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y)

In [4]:
X

,Bollinger_Upper,SMA_7,High,SMA_30,Low,Open,Bollinger_Lower,Close,Rolling_Mean,EMA_12,EMA_26,WMA,Adj Close,SMA_14
0,1191.651805,1027.857143,1070.0,1108.166667,1010.0,1070.0,982.848195,1070.0,1087.25,1058.693294,1079.837715,1041.181818,847.906067,1067.500000
1,1187.718224,1032.142857,1110.0,1107.500000,1060.0,1090.0,982.781776,1090.0,1085.25,1063.532787,1080.660585,1049.090909,863.754639,1062.500000
2,1174.631505,1042.142857,1090.0,1105.500000,1040.0,1050.0,983.868495,1050.0,1079.25,1061.442386,1078.194917,1049.909091,832.057068,1058.928571
3,1164.528794,1051.428571,1060.0,1102.500000,1010.0,1040.0,982.971206,1040.0,1073.75,1058.132256,1075.142715,1049.818182,824.132813,1056.071429
4,1155.917007,1051.428571,1030.0,1098.166667,995.0,1020.0,979.582993,1020.0,1067.75,1052.248758,1070.761766,1046.090909,808.284180,1052.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3426,10328.873676,10150.000000,10125.0,9923.333333,9950.0,10000.0,9671.126324,10100.0,10000.00,10084.187307,9974.022685,10140.454545,9872.500000,10037.500000
3427,10335.010121,10164.285714,10075.0,9936.666667,9925.0,10075.0,9684.989879,10075.0,10010.00,10082.773875,9981.502486,10128.181818,10075.000000,10053.571429
3428,10334.667075,10128.571429,10050.0,9953.333333,9950.0,10050.0,9707.832925,10050.0,10021.25,10077.731740,9986.576376,10112.272727,10050.000000,10071.428571
3429,10331.204055,10114.285714,10075.0,9967.500000,10000.0,10025.0,9738.795945,10075.0,10035.00,10077.311473,9993.126274,10102.727273,10075.000000,10092.857143


In [5]:
y

,Open,High,Low,Close
0,1070.0,1070.0,1010.0,1070.0
1,1090.0,1110.0,1060.0,1090.0
2,1050.0,1090.0,1040.0,1050.0
3,1040.0,1060.0,1010.0,1040.0
4,1020.0,1030.0,995.0,1020.0
...,...,...,...,...
3426,10000.0,10125.0,9950.0,10100.0
3427,10075.0,10075.0,9925.0,10075.0
3428,10050.0,10050.0,9950.0,10050.0
3429,10025.0,10075.0,10000.0,10075.0


In [6]:
X_scaled

array([[8.53761083e-03, 0.00000000e+00, 6.38977636e-03, ...,
        5.68839567e-04, 5.96459411e-03, 3.54107649e-03],
       [8.11107195e-03, 4.69079822e-04, 1.06496273e-02, ...,
        1.43706838e-03, 7.66874232e-03, 2.99024237e-03],
       [6.69201034e-03, 1.56359941e-03, 8.51970181e-03, ...,
        1.52688515e-03, 4.26039999e-03, 2.59678942e-03],
       ...,
       [9.99962802e-01, 9.96091001e-01, 9.62726305e-01, ...,
        9.96357431e-01, 9.95437456e-01, 9.95475291e-01],
       [9.99587288e-01, 9.94527402e-01, 9.65388711e-01, ...,
        9.95309568e-01, 9.98125629e-01, 9.97836009e-01],
       [9.99653596e-01, 9.93354703e-01, 9.68051118e-01, ...,
        9.94112011e-01, 9.98125629e-01, 1.00000000e+00]])

In [7]:
y_scaled

array([[0.00744681, 0.00638978, 0.00218341, 0.0075067 ],
       [0.00957447, 0.01064963, 0.00764192, 0.00965147],
       [0.00531915, 0.0085197 , 0.00545852, 0.00536193],
       ...,
       [0.96276596, 0.9627263 , 0.97816594, 0.97050938],
       [0.96010638, 0.96538871, 0.98362445, 0.97319035],
       [0.96010638, 0.96805112, 0.9808952 , 0.97319035]])

In [8]:
def create_sequences(X, y, time_steps=10):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        Xs.append(X[i:i+time_steps])
        ys.append(y[i+time_steps])
    return np.array(Xs), np.array(ys)

time_steps = 10


In [9]:
X_sequences,y_sequences = create_sequences(X_scaled,y_scaled,time_steps)

In [10]:
X_sequences.shape

(3421, 10, 14)

In [11]:
y_sequences.shape

(3421, 4)

In [12]:
train_size = round(len(X_scaled) * 0.80)
X_train = X_sequences[:train_size]
X_test = X_sequences[train_size:]
y_train = y_sequences[:train_size]
y_test = y_sequences[train_size:]

In [13]:
print(X_train.shape,X_test.shape)

(2745, 10, 14) (676, 10, 14)


In [14]:
print(y_train.shape,y_test.shape)

(2745, 4) (676, 4)


In [ ]:
from keras.layers import GRU, Dense, Dropout
from tensorflow.keras.losses import Huber
def build_model(input_shape, start_unit, num_layers, dropout_rate=0,output_units=1,asc=False):
  model = Sequential()
  for i in range(num_layers):
    if i == 0:
      model.add(GRU(units=start_unit, input_shape=input_shape, return_sequences=(i != num_layers - 1)))
    else:
      if asc==False:
        model.add(GRU(units=int(start_unit/(2*i)), return_sequences=(i != num_layers - 1)))
      else:
        model.add(GRU(units=int(start_unit*(2*i)), return_sequences=(i != num_layers - 1)))
    if dropout_rate > 0:
            model.add(Dropout(dropout_rate))
  model.add(Dense(output_units))
  model.compile(optimizer='adam', loss=Huber(), metrics=['mean_absolute_error'])
  return model

model_configs = [
  {'start_unit': 32, 'num_layers': 1, 'dropout_rate': 0,'asc':False}, #model 1L-1
  {'start_unit': 64, 'num_layers': 1, 'dropout_rate': 0,'asc':False}, #model 1L-2
  {'start_unit': 128, 'num_layers': 1, 'dropout_rate': 0,'asc':False},#model 1L-3
  {'start_unit': 256, 'num_layers': 1, 'dropout_rate': 0,'asc':False},#model 1L-4
  {'start_unit': 512, 'num_layers': 1, 'dropout_rate': 0,'asc':False},#model 1L-5
  {'start_unit': 32, 'num_layers': 2, 'dropout_rate': 0.2,'asc':True},#model 2LA-1
  {'start_unit': 256, 'num_layers': 2, 'dropout_rate': 0.2,'asc':True},#model 2LA-2
  {'start_unit': 64, 'num_layers': 2, 'dropout_rate': 0.2,'asc':False},#model 2LD-1
  {'start_unit': 512, 'num_layers': 2, 'dropout_rate': 0.2,'asc':False},#model 2LD-2
  {'start_unit': 32, 'num_layers': 3, 'dropout_rate': 0.2,'asc':True},#model 3LA-1
  {'start_unit': 128, 'num_layers': 3, 'dropout_rate': 0.2,'asc':True},#model 3LA-2
  {'start_unit': 128, 'num_layers': 3, 'dropout_rate': 0.2,'asc':False},#model 3LD-1
  {'start_unit': 512, 'num_layers': 3, 'dropout_rate': 0.2,'asc':False},#model 3LD-2
]
models = []
for config in model_configs:
    print(f"Training model with {config['start_unit']} GRU units, {config['num_layers']} layers, "
          f"and dropout rate {config['dropout_rate']}")

    model = build_model(input_shape=(X_train.shape[1],X_train.shape[2]),
                                    start_unit=config['start_unit'],
                                    num_layers=config['num_layers'],
                                    dropout_rate=config['dropout_rate'],
                                    asc = config['asc'],
                                    output_units=y_train.shape[1])
    # Latih model
    early_stop = EarlyStopping(monitor='loss',patience=10)
    history = model.fit(X_train, y_train, epochs=100, batch_size=32,verbose=1)

    # Simpan model
    model_name = f"gru_model_{config['start_unit']}_units_{config['num_layers']}_layers_dropout_{config['dropout_rate']}_asc_{config['asc']}.h5"
    model.save(model_name)

    # Simpan model dan history
    models.append({'model': model, 'history': history, 'config': config})

Training model with 32 GRU units, 1 layers, and dropout rate 0
Epoch 1/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 0.0209 - mean_absolute_error: 0.1135
Epoch 2/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 1.2617e-04 - mean_absolute_error: 0.0111
Epoch 3/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 7.2905e-05 - mean_absolute_error: 0.0073
Epoch 4/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 6.9992e-05 - mean_absolute_error: 0.0072
Epoch 5/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 5.6525e-05 - mean_absolute_error: 0.0066
Epoch 6/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 5.2067e-05 - mean_absolute_error: 0.0065
Epoch 7/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 5.6924e-05 - mean_absolute_error: 0.0065
Epoch 8/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 4.6925e-05 - mean_absolute_error: 0.0060
Epoch 9/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 4.9177e-05 - mean_absolute_error: 0.0060
Epoch 10/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 0

Training model with 64 GRU units, 1 layers, and dropout rate 0
Epoch 1/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 0.0169 - mean_absolute_error: 0.1070
Epoch 2/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 8.5935e-05 - mean_absolute_error: 0.0089
Epoch 3/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 7.5942e-05 - mean_absolute_error: 0.0079
Epoch 4/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 7.0642e-05 - mean_absolute_error: 0.0075
Epoch 5/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 6.9916e-05 - mean_absolute_error: 0.0075
Epoch 6/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 6.5669e-05 - mean_absolute_error: 0.0071
Epoch 7/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 5.4799e-05 - mean_absolute_error: 0.0068
Epoch 8/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 5.7700e-05 - mean_absolute_error: 0.0066
Epoch 9/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 5.5233e-05 - mean_absolute_error: 0.0065
Epoch 10/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 0

Training model with 128 GRU units, 1 layers, and dropout rate 0
Epoch 1/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0078 - mean_absolute_error: 0.0670
Epoch 2/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 5.2873e-05 - mean_absolute_error: 0.0065
Epoch 3/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 5.5459e-05 - mean_absolute_error: 0.0065
Epoch 4/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 4.7724e-05 - mean_absolute_error: 0.0062
Epoch 5/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 4.5098e-05 - mean_absolute_error: 0.0060
Epoch 6/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 4.0263e-05 - mean_absolute_error: 0.0058
Epoch 7/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 3.4292e-05 - mean_absolute_error: 0.0055
Epoch 8/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 3.7446e-05 - mean_absolute_error: 0.0055
Epoch 9/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 3.6118e-05 - mean_absolute_error: 0.0054
Epoch 10/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 

Training model with 256 GRU units, 1 layers, and dropout rate 0
Epoch 1/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0066 - mean_absolute_error: 0.0596
Epoch 2/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 6.4325e-05 - mean_absolute_error: 0.0068
Epoch 3/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 6.4744e-05 - mean_absolute_error: 0.0069
Epoch 4/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 5.0211e-05 - mean_absolute_error: 0.0063
Epoch 5/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 4.7933e-05 - mean_absolute_error: 0.0063
Epoch 6/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 4.1878e-05 - mean_absolute_error: 0.0057
Epoch 7/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 4.2038e-05 - mean_absolute_error: 0.0058
Epoch 8/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 3.4084e-05 - mean_absolute_error: 0.0053
Epoch 9/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 3.5915e-05 - mean_absolute_error: 0.0055
Epoch 10/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 

Training model with 512 GRU units, 1 layers, and dropout rate 0
Epoch 1/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 0.0060 - mean_absolute_error: 0.0542
Epoch 2/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 5.6027e-05 - mean_absolute_error: 0.0065
Epoch 3/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 5.3619e-05 - mean_absolute_error: 0.0066
Epoch 4/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 4.9732e-05 - mean_absolute_error: 0.0063
Epoch 5/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 5.4377e-05 - mean_absolute_error: 0.0067
Epoch 6/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 3.6766e-05 - mean_absolute_error: 0.0054
Epoch 7/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 3.2424e-05 - mean_absolute_error: 0.0052
Epoch 8/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 3.7940e-05 - mean_absolute_error: 0.0056
Epoch 9/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 3.2241e-05 - mean_absolute_error: 0.0052
Epoch 10/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 

...

Training model with 512 GRU units, 2 layers, and dropout rate 0.2
Epoch 1/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 0.0053 - mean_absolute_error: 0.0549
Epoch 2/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 5.0673e-04 - mean_absolute_error: 0.0208
Epoch 3/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 3.1617e-04 - mean_absolute_error: 0.0166
Epoch 4/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 2.8130e-04 - mean_absolute_error: 0.0155
Epoch 5/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 3.0488e-04 - mean_absolute_error: 0.0163
Epoch 6/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 2.4860e-04 - mean_absolute_error: 0.0145
Epoch 7/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 2.2975e-04 - mean_absolute_error: 0.0141
Epoch 8/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 3.0166e-04 - mean_absolute_error: 0.0167
Epoch 9/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 2.1606e-04 - mean_absolute_error: 0.0135
Epoch 10/100
86/86 ━━━━━━━━━━━━━━━━━━━

Training model with 32 GRU units, 3 layers, and dropout rate 0.2
Epoch 1/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - loss: 0.0099 - mean_absolute_error: 0.0823
Epoch 2/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 8.6296e-04 - mean_absolute_error: 0.0270
Epoch 3/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 7.0068e-04 - mean_absolute_error: 0.0244
Epoch 4/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 6.4097e-04 - mean_absolute_error: 0.0238
Epoch 5/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 5.2507e-04 - mean_absolute_error: 0.0219
Epoch 6/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 4.3995e-04 - mean_absolute_error: 0.0193
Epoch 7/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 4.1693e-04 - mean_absolute_error: 0.0195
Epoch 8/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 3.7137e-04 - mean_absolute_error: 0.0184
Epoch 9/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 3.6938e-04 - mean_absolute_error: 0.0179
Epoch 10/100
86/86 ━━━━━━━━━━━━━━━━━━

Training model with 128 GRU units, 3 layers, and dropout rate 0.2
Epoch 1/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - loss: 0.0073 - mean_absolute_error: 0.0648
Epoch 2/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 3.4974e-04 - mean_absolute_error: 0.0173
Epoch 3/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 3.1162e-04 - mean_absolute_error: 0.0165
Epoch 4/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 2.5248e-04 - mean_absolute_error: 0.0147
Epoch 5/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 3.5839e-04 - mean_absolute_error: 0.0178
Epoch 6/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 2.7695e-04 - mean_absolute_error: 0.0158
Epoch 7/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 2.0210e-04 - mean_absolute_error: 0.0134
Epoch 8/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 2.3780e-04 - mean_absolute_error: 0.0147
Epoch 9/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 2.4519e-04 - mean_absolute_error: 0.0149
Epoch 10/100
86/86 ━━━━━━━━━━━━━━━━━━

Training model with 128 GRU units, 3 layers, and dropout rate 0.2
Epoch 1/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 0.0093 - mean_absolute_error: 0.0840
Epoch 2/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0018 - mean_absolute_error: 0.0383
Epoch 3/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.0014 - mean_absolute_error: 0.0334
Epoch 4/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0011 - mean_absolute_error: 0.0301
Epoch 5/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.0010 - mean_absolute_error: 0.0287
Epoch 6/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 8.4993e-04 - mean_absolute_error: 0.0267
Epoch 7/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 7.6133e-04 - mean_absolute_error: 0.0250
Epoch 8/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 6.9814e-04 - mean_absolute_error: 0.0240
Epoch 9/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 6.7217e-04 - mean_absolute_error: 0.0240
Epoch 10/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/ste

Training model with 512 GRU units, 3 layers, and dropout rate 0.2
Epoch 1/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - loss: 0.0051 - mean_absolute_error: 0.0604
Epoch 2/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 7.3497e-04 - mean_absolute_error: 0.0254
Epoch 3/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 5.6985e-04 - mean_absolute_error: 0.0221
Epoch 4/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 5.5541e-04 - mean_absolute_error: 0.0221
Epoch 5/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 4.5913e-04 - mean_absolute_error: 0.0201
Epoch 6/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 4.2384e-04 - mean_absolute_error: 0.0193
Epoch 7/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 4.9006e-04 - mean_absolute_error: 0.0209
Epoch 8/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 3.7135e-04 - mean_absolute_error: 0.0175
Epoch 9/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 4.1273e-04 - mean_absolute_error: 0.0186
Epoch 10/100
86/86 ━━━━━━━━━━━━━━━━━

In [ ]:
# title=' Loss and Mean Absolute Error vs. Epochs '
# xlabel=' Epochs '
# GRU_losses = pd.DataFrame(GRU_Model1.history.history)


# ax = GRU_losses.plot(figsize=(10,6),title=title)
# ax.autoscale(axis='x',tight=True)
# ax.set(xlabel=xlabel);




In [ ]:
# def predict_and_inverse_transform2(DF, X_test, model, scaler,y):
#     # Salin data tes terakhir sebanyak len(X_test) baris
#     test = DF.iloc[-len(X_test):].copy()

#     # Lakukan prediksi menggunakan model

#     predictions = model.predict(X_test)

#     # Hanya lakukan inverse transform pada prediksi (misalnya hanya pada 4 kolom prediksi)
#     inverse_predictions = scaler.inverse_transform(predictions)

#     # Buat dataframe dari prediksi ter-inverse dengan index yang sama seperti data tes
#     inverse_predictions = pd.DataFrame(inverse_predictions,
#                                        columns=['Predicted Open','Predicted High', 'Predicted Low', 'Predicted Close'],
#                                        index=test.index)

#     # Gabungkan prediksi dengan data tes asli
#     test_df = y.iloc[-len(X_test):].copy()

#     # Jika ingin melakukan inverse transform pada data asli (hanya kolom yang di-scale):
#     test_df = scaler.inverse_transform(test_df)

#     return test_df,inverse_predictions


In [ ]:
def inverse_predict(model):
    predictions = model.predict(X_test)
    inverse_predictions = scaler_y.inverse_transform(predictions)
    inverse_predictions = pd.DataFrame(inverse_predictions,
                                       columns=['Predicted Open', 'Predicted High', 'Predicted Low', 'Predicted Close'])
    test_df = y_test
    test_df = scaler_y.inverse_transform(test_df)
    test_df = pd.DataFrame(test_df, columns=['Open', 'High', 'Low', 'Close'])
    return test_df, inverse_predictions

In [ ]:
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# Modifikasi fungsi untuk menambahkan hasil evaluasi ke df_evaluation dan menyimpannya ke CSV
def evaluate_metrics(test_df, inverse_predictions, actual, predicted, model_name, df_evaluation):
    rmse = np.sqrt(mean_squared_error(test_df[actual], inverse_predictions[predicted]))
    mae = mean_absolute_error(test_df[actual], inverse_predictions[predicted])
    mape = np.mean(np.abs((test_df[actual] - inverse_predictions[predicted]) / test_df[actual])) * 100
    r2 = r2_score(test_df[actual], inverse_predictions[predicted])

    # Simpan hasil ke dataframe menggunakan pd.concat()
    new_row = pd.DataFrame({
        'Model': [model_name],
        'Variable': [f'{actual}'],
        'RMSE': [rmse],
        'MAE': [mae],
        'MAPE': [mape],
        'R2 Score': [r2]
    })

    df_evaluation = pd.concat([df_evaluation, new_row], ignore_index=True)

    return df_evaluation

# Inisialisasi dataframe untuk menyimpan hasil evaluasi
df_evaluation = pd.DataFrame(columns=['Model', 'Variable', 'RMSE', 'MAE', 'MAPE', 'R2 Score'])

# Loop untuk model dan simpan hasil evaluasi
for model_dict in models:
    model_name = model_dict['config']
    print(f"Evaluating model with config: {model_name}")
    test_df, inverse_predictions = inverse_predict(model_dict['model'])

    print(f"Close Price Evaluation")
    df_evaluation = evaluate_metrics(test_df, inverse_predictions, actual='Close', predicted='Predicted Close', model_name=model_name, df_evaluation=df_evaluation)
    df_evaluation = evaluate_metrics(test_df, inverse_predictions, actual='Open', predicted='Predicted Open', model_name=model_name, df_evaluation=df_evaluation)
    df_evaluation = evaluate_metrics(test_df, inverse_predictions, actual='High', predicted='Predicted High', model_name=model_name, df_evaluation=df_evaluation)
    df_evaluation = evaluate_metrics(test_df, inverse_predictions, actual='Low', predicted='Predicted Low', model_name=model_name, df_evaluation=df_evaluation)
# Simpan dataframe evaluasi ke CSV
df_evaluation.to_csv(f'{var_bank}_model_evaluation_metrics.csv', index=False, decimal=',')
print("Evaluation metrics saved to model_evaluation_metrics.csv")


In [ ]:
tes = pd.read_csv(f'/content/{var_bank}_model_evaluation_metrics.csv')
tes

In [ ]:
from tensorflow.keras.models import load_model

# Load model
model = load_model('/content/gru_model_64_units_1_layers_dropout_0_asc_False.h5')


In [ ]:
test_df, inverse_predictions = inverse_predict(model)

In [ ]:
tanggal = pd.to_datetime(df['Date'], errors='coerce', format= '%Y-%m-%d')
tanggal = tanggal[train_size+10:]
tanggal

In [ ]:
def calculate_mape_per_row(actual, predicted):
    actual, predicted = np.array(actual), np.array(predicted)
    mape_per_row = np.abs((actual - predicted) / actual) * 100
    return mape_per_row
mape_per_row = calculate_mape_per_row(test_df, inverse_predictions)
mape_per_row

In [ ]:
mape = pd.DataFrame(mape_per_row, columns=['Mape Open', 'Mape Close', 'Mape High', 'Mape Low'])
combined_df = pd.concat([test_df, inverse_predictions, mape], axis=1)
combined_df

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Konversi tanggal
tanggal = pd.to_datetime(df['Date'], errors='coerce', format='%Y-%m-%d')
tanggal = tanggal[train_size  + 10:]

# Pastikan panjang `tanggal` cocok dengan `test_df` dan `inverse_predictions`
test_df = test_df.set_index(tanggal)
inverse_predictions = inverse_predictions.set_index(tanggal)
# Plot
plt.figure(figsize=(10, 6))
test_df['Close'].plot(label='Close Price').autoscale(axis='x', tight=True)
inverse_predictions['Predicted Close'].plot(label='Predicted Close Price').autoscale(axis='x', tight=True)

plt.legend()
plt.title('Comparison of Actual and Predicted Close Prices', backgroundcolor='grey', color='white', fontweight='bold')
plt.xlabel('Date', labelpad=15)
plt.ylabel('Price', labelpad=15)
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Konversi tanggal
tanggal = pd.to_datetime(df['Date'], errors='coerce', format='%Y-%m-%d')
tanggal = tanggal[train_size + 10:]

# Pastikan panjang `tanggal` cocok dengan `test_df` dan `inverse_predictions`
test_df = test_df.set_index(tanggal)
inverse_predictions = inverse_predictions.set_index(tanggal)

# Plot
plt.figure(figsize=(10, 6))
test_df['Open'].plot(label='Open Price').autoscale(axis='x', tight=True)
inverse_predictions['Predicted Open'].plot(label='Predicted Open Price').autoscale(axis='x', tight=True)

plt.legend()
plt.title('Comparison of Actual and Predicted Open Prices', backgroundcolor='grey', color='white', fontweight='bold')
plt.xlabel('Date', labelpad=15)
plt.ylabel('Price', labelpad=15)
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Konversi tanggal
tanggal = pd.to_datetime(df['Date'], errors='coerce', format='%Y-%m-%d')
tanggal = tanggal[train_size  + 10:]

# Pastikan panjang `tanggal` cocok dengan `test_df` dan `inverse_predictions`
test_df = test_df.set_index(tanggal)
inverse_predictions = inverse_predictions.set_index(tanggal)

# Plot
plt.figure(figsize=(10, 6))
test_df['High'].plot(label='High Price').autoscale(axis='x', tight=True)
inverse_predictions['Predicted High'].plot(label='Predicted High Price').autoscale(axis='x', tight=True)

plt.legend()
plt.title('Comparison of Actual and Predicted High Prices', backgroundcolor='grey', color='white', fontweight='bold')
plt.xlabel('Date', labelpad=15)
plt.ylabel('Price', labelpad=15)
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Konversi tanggal
tanggal = pd.to_datetime(df['Date'], errors='coerce', format='%Y-%m-%d')
tanggal = tanggal[train_size + 10:]

# Pastikan panjang `tanggal` cocok dengan `test_df` dan `inverse_predictions`
test_df = test_df.set_index(tanggal)
inverse_predictions = inverse_predictions.set_index(tanggal)

# Plot
plt.figure(figsize=(10, 6))
test_df['Low'].plot(label='Low Price').autoscale(axis='x', tight=True)
inverse_predictions['Predicted Low'].plot(label='Predicted Low Price').autoscale(axis='x', tight=True)

plt.legend()
plt.title('Comparison of Actual and Predicted Low Prices', backgroundcolor='grey', color='white', fontweight='bold')
plt.xlabel('Date', labelpad=15)
plt.ylabel('Price', labelpad=15)
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots
tgl = df['Date']
prediksi_high = inverse_predictions['Predicted High']
prediksi_low = inverse_predictions['Predicted Low']
prediksi_close = inverse_predictions['Predicted Close']
prediksi_open = inverse_predictions['Predicted Open']
fig = make_subplots(rows=4, cols=1)
fig.add_trace(go.Scatter(x=tgl, y=df['High'], name='Actual High'),row=1,col=1)
fig.add_trace(go.Scatter(x=tgl[train_size:], y=prediksi_high, name='Predicted high'),row=1,col=1)
fig.add_trace(go.Scatter(x=tgl, y=df['Low'], name='Actual Low'),row=2,col=1)
fig.add_trace(go.Scatter(x=tgl[train_size:], y=prediksi_low, name='Predicted Low'),row=2,col=1)
fig.add_trace(go.Scatter(x=tgl, y=df['Open'], name='Actual Open'),row=3,col=1)
fig.add_trace(go.Scatter(x=tgl[train_size:], y=prediksi_open, name='Predicted Open'),row=3,col=1)
fig.add_trace(go.Scatter(x=tgl, y=df['Close'], name='Actual Close'),row=4,col=1)
fig.add_trace(go.Scatter(x=tgl[train_size:], y=prediksi_close, name='Predicted close'),row=4,col=1)


fig.show()